In [3]:
%%capture
from tqdm import tqdm_notebook as tqdm
tqdm().pandas()

import requests
import numpy as np
import re
import string
from gensim.models import Word2Vec

import nltk
from nltk.corpus import gutenberg, stopwords
from nltk.collocations import *
from nltk import FreqDist, word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

## Acquiring User Likes


### Attempting through Twitter API -- See 1.2 for code that was finally used

In [153]:
### Acquired from Twitter using API key and API secret key ###
bearer_token = 'AAAAAAAAAAAAAAAAAAAAAImyAwEAAAAAyWB9oAfMqhJG66%2BeJHuUEi5wsXM%3DKOzTDPgjqoXjzERO1JmoAJfVIqjnaPdD7wUhayNjESzeytty2J' # Bearer Token


In [157]:
def get_liked_tweets(handle):
    '''
    Takes Twitter handle and extracts and parses liked tweets.
    
    '''
    all_tweets = []
    url = 'https://api.twitter.com/1.1/favorites/list.json?screen_name={}'.format(handle)
    headers = {
        'Authorization': 'Bearer {}'.format(bearer_token),
            }
    first_url_params = {
        'count': 200
            }
    first_response = requests.get(url, headers=headers, params=first_url_params)
    first_initial_split = first_response.text.split('"text":"')
    first_t = len(first_initial_split)
    for i in range(1,first_t):  
        all_tweets.append(first_initial_split[i].split(',"truncated":')[0])
    
    ### Regex to extract the id of the last tweet to use as pagination ###      
    regsearch = re.search('(?<=(,"id":))[\d]*', first_initial_split[first_t-2])
    tweet_id = int(regsearch.group(0))
    ###
    
    ### Looping process to manually paginate through tweets ###
    page = 0
    while page < 75:
        url_params = {
        'count': 200,
        'max_id': tweet_id-1
            }
        response = requests.get(url, headers=headers, params=url_params)
        initial_split = response.text.split('"text":"')
        t = len(initial_split)
        print("attempting another one")
        ### Regex to extract the id of the last tweet to use as pagination ###      
        regsearch = re.search('(?<=(,"id":))[\d]*', initial_split[t-2])
        tweet_id = int(regsearch.group(0))
        ###
        for i in range(1, t):
            all_tweets.append(initial_split[i].split(',"truncated":')[0])
        page +=1

    return all_tweets

In [155]:
twitter_handle = input("What is your Twitter Handle?")    ### Commented so I don't have to re-type the handle

What is your Twitter Handle?Brit_Michele_


In [158]:
liked_tweets = get_liked_tweets(twitter_handle)

attempting another one
attempting another one
attempting another one
attempting another one
attempting another one
attempting another one
attempting another one
attempting another one
attempting another one
attempting another one
attempting another one
attempting another one
attempting another one
attempting another one
attempting another one
attempting another one
attempting another one


AttributeError: 'NoneType' object has no attribute 'group'

In [31]:
data = liked_tweets.map(word_tokenize)

NameError: name 'liked_tweets' is not defined

In [78]:
model = Word2Vec([liked_tweets],size=100,window=5,min_count=3,workers=4)

In [79]:
model.train([liked_tweets],total_examples=model.corpus_count,epochs=10)

(100000, 3779140)

In [146]:
wv = model.wv

In [145]:
wv.most_similar('harry')

KeyError: "word 'harry' not in vocabulary"

### Attempting through Twint

In [273]:
import twint
import nest_asyncio
import pandas as pd
from nltk import word_tokenize
nest_asyncio.apply()

In [275]:
# Configure
c = twint.Config()
c.Username = 'Brit_Michele_'
c.Pandas = True
c.Limit = 30000
tqdm(twint.run.Favorites(c))

likes_df = twint.storage.panda.Tweets_df

1195042530175463424 2019-11-14 13:15:16 EST <DrizzyJayyyy> *Spends full day wt her*   Me: “I’m about to go home”  Her: pic.twitter.com/eNKYOq98e3
1194829694085914624 2019-11-13 23:09:32 EST <machinegunkelly> this little flower came up to me and said i didn’t play her favorite song at the show so ... pic.twitter.com/QEgszXSP4S
1197695398921613312 2019-11-21 20:56:49 EST <eveyhammond> I want someone else to go through my purse one day and tell me what the hell is in there
1194819963716067329 2019-11-13 22:30:52 EST <thedapperealist> Women may not do a lot at the beginning of courting but once they like you like you it’s over. Anything you want is yours.
1197603905699491840 2019-11-21 14:53:16 EST <snooki> Im screaming https://twitter.com/tabithakuehn/status/1196660747205525508 …
1194782591506759680 2019-11-13 20:02:22 EST <ItsDanSheehan> Hey welcome to being alive! In high school you’re going to date one (1) fifteen year old person who will determine your comfort level with healthy intim

1194394714469998593 2019-11-12 18:21:05 EST <QuirkyChe> When an embarrassing moment from years ago pops into your head at random times: https://twitter.com/VictoriousOoC/status/1194352353773666304 …
1196226857341005824 2019-11-17 19:41:22 EST <AstrologyVibez> Being in love with a Leo:
1194374297151668224 2019-11-12 16:59:57 EST <Mclovviin_> No one :  Me when my boo texts me : pic.twitter.com/lhCndRxN30
1196178107746717696 2019-11-17 16:27:39 EST <kaitlynmidgett> How exactly does Harry Styles do that?
1194364818960080898 2019-11-12 16:22:17 EST <CaucasianJames> if i was the kid in home alone i would’ve just cried and walked to my neighbors and the movie would’ve been over
1194358318803771392 2019-11-12 15:56:27 EST <ellorysmith> my cat is on antibiotics and for one second my fucking brain was like “wait this means he can’t drink”
1196164086469296128 2019-11-17 15:31:56 EST <tarrascoo> me talking                     when  about how                     someone else hot my                 

1193709768998825986 2019-11-10 20:59:21 EST <HOIAsburyPark> Your mom’s got great taste, lilpissbaby. We can see where you get it from. https://twitter.com/lilpissbaby/status/1192650349049516032 …
1194782591506759680 2019-11-13 20:02:22 EST <ItsDanSheehan> Hey welcome to being alive! In high school you’re going to date one (1) fifteen year old person who will determine your comfort level with healthy intimacy for the rest of your adult life!
1194767840500281344 2019-11-13 19:03:45 EST <kaitlynmidgett> I might be single and also depressed but at least I have my own snack drawer at work
1193702900800741376 2019-11-10 20:32:04 EST <postgrad_barty> just found out my best friend from the bathroom bar line two years ago unfollowed me on Instagram... shocked and confused... no one talk to me in this trying time
1193650813996064776 2019-11-10 17:05:05 EST <kaitlynmidgett> Me and @Brit_Michele_ as fuck https://twitter.com/luxykendall/status/1193626103350513664 …
1194763197783277569 2019-11-13 18

1192208048602976257 2019-11-06 17:32:03 EST <bIondiewasabi> women will roast men for not washing their sheets for a few months and then go 12,000 miles without an oil change
1194336400616779777 2019-11-12 14:29:22 EST <richeepeace> nobody:  this guy at edc orlando: everyone here is tripping BALLS, so lemme fuck em up real quickk pic.twitter.com/NV7zBYAMpo
1192185619260354560 2019-11-06 16:02:56 EST <BruceWiegner> Malicious Music Video out now! 😈  https://youtu.be/abMGlUKCwOA   Directed by Cory Pitkavish pic.twitter.com/IOa7BWfpzr
1194333156683517952 2019-11-12 14:16:28 EST <necessaryaf> drunk me is so determined to embarrass me.. I'm so good to her, I put her in cute little outfits and and get her ready and what does she do???? she ruins me .
1192142187296804864 2019-11-06 13:10:21 EST <turtlebreezee> I underestimate the power of letting go every time I hold on
1194317311169024000 2019-11-12 13:13:30 EST <yxngpvp1> It’s cold as FUCK... my anemic ass is not having fun rn
119431428998684

1191467332616503302 2019-11-04 16:28:43 EST <SandyCandy42> @Brit_Michele_ im lookin at you
1193233152430542851 2019-11-09 13:25:27 EST <DrizzyJayyyy> when you’re soft as fuck but you had enough pic.twitter.com/DRCZMjwrhI
1191455636057395208 2019-11-04 15:42:14 EST <picshs> HARRY STYLES.              FINE LINE 2017.                                 2019. pic.twitter.com/VHh4tueNDU
1191438649306558464 2019-11-04 14:34:44 EST <earthdombaby> Fixed signs (Taurus-Leo-Scorpio-Aqua) have a long/hate relationship with routine - they need a consistent schedule in order to accomplish their goals, but they also feel as though it can weigh them down when they want to step out of that.
1193230138315460608 2019-11-09 13:13:28 EST <OregonProgress> Cinderblock doing a full U turn before pouncing. pic.twitter.com/KLCd8xxpVt
1191418969271521288 2019-11-04 13:16:32 EST <socratestyIes> harry will not be ‘coming out’ with this album. he is already non-labeling. please do not discredit him or anyone else who 

...

1179057304836005888 2019-10-01 11:35:41 EST <MODSUN> I love u today. More than yesterday,  But not as much as tomorrow.
1178520357977776128 2019-09-30 00:02:03 EST <alexandraerd_> People don’t tell you that just because you get out of a toxic relationship, that doesn’t mean that person can’t affect you anymore.
1179056945837285378 2019-10-01 11:34:16 EST <maggiexmah> I’m on the subway, it feels like I’m in a greys anatomy episode pic.twitter.com/33q6XntYes
1178439807296512005 2019-09-29 18:41:59 EST <jedwardofficial> sean kingston really said these women are so beautiful i want to kill myself
1179046612921933824 2019-10-01 10:53:12 EST <itsdougthepug> Me waking up on October 1st 🎃 pic.twitter.com/sol7CWQUVK
1178428293378428935 2019-09-29 17:56:13 EST <elizardd> Haunted houses aren’t scary when you realize it’s just a bunch of theater kids in there
1179044051175329794 2019-10-01 10:43:02 EST <BugGirlOfficial> over 90° on october 1st what the fuck r y’all doing to this planet
11784129389

1178139201847484416 2019-09-28 22:47:29 EST <noahcent> *Photoshops self into significant others memories*
1176736001009623040 2019-09-25 01:51:39 EST <HeavenSentMel> First of all You have to deserve it https://twitter.com/zenisthekey/status/1175915533608394754 …
1178107529374633984 2019-09-28 20:41:37 EST <stfudecade> me : i’m really done w him i stg idc fr   him : i miss you   me : i miss you more 🥺🥺🥺
1176658957026217984 2019-09-24 20:45:31 EST <JbKnockout> Why was this technology abandoned https://twitter.com/WhoaInteresting/status/1152595595976204293 …
1177808350035226624 2019-09-28 00:52:47 EST <EdenSemel> Can’t wait til boys are 25..........and that frontal lobe develops.... mmmmm
1177808296805199872 2019-09-28 00:52:35 EST <milkstrology> new moon 9/28 is good for.. 🌟 aries: self-care taurus: creative hobbies  gemini: h u s t l e cancer: flirt leo: releasing emotional baggage virgo: trying new things libra: $$ moves scorpio: socializing  sag: healing cap: getting off couch aqua: g

1174882558397550593 2019-09-19 23:06:44 EST <Bahia_kanum> Shit man...people be like 'Netflix heals depression' 'pizza makes you happy' 'sex eases the pain'. Motherfucker have you ever watched David dobrik! That shits therapy... @DavidDobrik
1173681239729029121 2019-09-16 15:33:08 EST <dxxnya> if you don’t believe in male privilege let me just point out the fact that they sell boner pills at 7/11. you know what happens to us if we can’t get physically aroused? chad spits on our vagina, that’s what
1173630618128867332 2019-09-16 12:11:59 EST <lilpissbaby> #NewProfilePic pic.twitter.com/jHqXYArXYu
1174860976950251527 2019-09-19 21:40:59 EST <whoreacio> i WILL vine reference my way into your heart
1173580007844851714 2019-09-16 08:50:52 EST <fairycrust> the school nurse giving me ginger ale and saltines to fix my sprained ankle pic.twitter.com/aydMHX4U7t
1174818334317133827 2019-09-19 18:51:32 EST <kaitlynmidgett> Me being protective over @Brit_Michele_ https://twitter.com/DailyMail/status

1172854454628077568 2019-09-14 08:47:47 EST <sagistarbb> The sign your Saturn sits in is the sign you understand in terms of their negative qualities. You know why they do the shitty things they do and what they do to do it, & because of this, you often end up either resenting or parenting them. (Thread)
1171828094270881799 2019-09-11 12:49:23 EST <salaransarii> STOP talking shit about white rappers  G-Eazy  Post Malone   Logic  Mac Miller is UNIQUE  Eminem   Machine Gun Kelly  Macklemore   Lil Dicky
1172687031291527169 2019-09-13 21:42:30 EST <sfergs_> Our healthcare is free. We don’t have $50,000 student loans at age 18. Medicine is free for youth under 25. Marijuana is legal. Abortions are free. Our drinking age is 19. Your president is a racist orange cheeto & your police kill black people for sport. Sit this one out, love. https://twitter.com/fredsonora/status/1172685276071641088 …
1171693540856012800 2019-09-11 03:54:43 EST <HANGEDD6LL> get the pattern, time passages, time nomad,

KeyboardInterrupt: 

In [138]:
data = likes_df['tweet']

In [139]:
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

[nltk_data] Downloading package wordnet to /Users/reuben/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [140]:
import spacy
spacy.load('en')
from spacy.lang.en import English
parser = English()

def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

In [141]:
nltk.download('stopwords')
stopwords = set(nltk.corpus.stopwords.words('english'))


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/reuben/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [264]:
def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 3]
    tokens = [token for token in tokens if token not in stopwords]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

In [265]:
import random
text_data = []
for line in data:
    tokens = prepare_text_for_lda(line)
    if random.random() > .99:
        print(tokens)
        text_data.append(tokens)

NameError: name 'data' is not defined

In [144]:
from gensim import corpora
dictionary = corpora.Dictionary(text_data)
corpus = [dictionary.doc2bow(text) for text in text_data]
import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

In [150]:
import gensim
NUM_TOPICS = 10
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=300)
ldamodel.save('model5.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.083*"things" + 0.083*"pic.twitter.com/jg3ayhs2k9" + 0.083*"explain" + 0.083*"anywhere"')
(1, '0.233*"bitch" + 0.083*"sensitive" + 0.008*"wanna" + 0.008*"actually"')
(2, '0.011*"friend" + 0.011*"hire" + 0.011*"thanks" + 0.011*"actually"')
(3, '0.040*"things" + 0.040*"humans" + 0.040*"control" + 0.040*"understand"')
(4, '0.049*"money" + 0.049*"split" + 0.049*"please" + 0.049*"place"')
(5, '0.077*"someone" + 0.077*"thanks" + 0.077*"heart" + 0.077*"solid"')
(6, '0.067*"least" + 0.067*"steve" + 0.067*"reunion" + 0.067*"dustin"')
(7, '0.103*"relationship" + 0.103*"care" + 0.054*"someone" + 0.054*"mental"')
(8, '0.043*"going" + 0.043*"someone" + 0.022*"start" + 0.022*"matter"')
(9, '0.067*"enjoy" + 0.067*"summer" + 0.067*"pic.twitter.com/egztopntuf" + 0.067*"canela"')


## Getting list of top subreddits

In [5]:
driver = webdriver.Chrome()
driver.get('https://www.reddit.com/r/TheoryOfReddit/comments/1f7hqc/the_200_most_active_subreddits_categorized_by/')

In [6]:
subs = driver.find_elements_by_class_name('_1qeIAgB0cPwnLhDF9XSiJM')

In [7]:
subreddits = []
for i in subs[3:]:
    if "r/" in i.text:
        subreddits.append(i.text.split(" - ")[1])
    else:
        continue

IndexError: list index out of range

In [8]:
clean_subs = []
for sub in subreddits:
    clean_subs.append(sub.split('r/')[1])

## Using PRAW (Python Reddit API Wrapper) to get top posts in top subreddits

In [1]:
import praw
reddit = praw.Reddit(client_id='MU9yce7zmiOKwA',
                     client_secret='DRV5fO7p-Tn7VzxmMH8mnRwM_hE',
                     user_agent='my user agent')

In [11]:
all_submissions = {}
for sub in tqdm(clean_subs):
    all_submissions[sub] = []
    for submission in reddit.subreddit(sub).top(limit=1000):
        all_submissions[sub].append(submission.title)
    

In [33]:
lemmatizer = WordNetLemmatizer()

pattern = "([a-zA-Z]+(?:'[a-z]+)?)"
subr_text_tokens_raw = nltk.regexp_tokenize(" ".join(all_submissions['creepy']), pattern)
subr_text_tokens = [word.lower() for word in subr_text_tokens_raw]
subr_text_tokens = [lemmatizer.lemmatize(i) for i in subr_text_tokens]
subr_text_freqdist = FreqDist(subr_text_tokens)
subr_text_freqdist.most_common(200)

[('the', 319),
 ('a', 216),
 ('of', 174),
 ('in', 166),
 ('by', 93),
 ('to', 79),
 ('i', 75),
 ('this', 72),
 ('my', 68),
 ('from', 56),
 ('and', 55),
 ('on', 41),
 ('at', 35),
 ('for', 30),
 ('wa', 30),
 ('with', 30),
 ('skull', 29),
 ('it', 28),
 ('found', 27),
 ('me', 26),
 ('an', 26),
 ('is', 23),
 ('that', 22),
 ('r', 22),
 ('night', 20),
 ('like', 20),
 ('out', 19),
 ('s', 19),
 ('man', 18),
 ('old', 18),
 ('just', 16),
 ('death', 16),
 ('year', 16),
 ('human', 16),
 ('post', 15),
 ('creepy', 14),
 ('bone', 14),
 ('one', 14),
 ('picture', 14),
 ('look', 14),
 ('made', 14),
 ('mask', 14),
 ('when', 14),
 ('his', 13),
 ('you', 13),
 ('skeleton', 13),
 ('friend', 12),
 ('abandoned', 12),
 ('monster', 12),
 ('statue', 12),
 ('x', 12),
 ('no', 11),
 ('some', 11),
 ('what', 11),
 ('wood', 11),
 ('into', 11),
 ('water', 11),
 ('people', 11),
 ('face', 11),
 ('last', 11),
 ('thing', 11),
 ('inside', 10),
 ('back', 10),
 ('house', 10),
 ('under', 10),
 ('dead', 10),
 ('church', 10),
 ('af

In [34]:
stopwords_list = stopwords.words('english')
stopwords_list += list(string.punctuation)
stopwords_list += ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'wa','nsfw']

subr_text_stopped = [word for word in subr_text_tokens if word not in stopwords_list]

In [35]:
subr_text_stopped_freqdist = FreqDist(subr_text_stopped)
subr_text_stopped_freqdist.most_common(50)

[('skull', 29),
 ('found', 27),
 ('r', 22),
 ('night', 20),
 ('like', 20),
 ('man', 18),
 ('old', 18),
 ('death', 16),
 ('year', 16),
 ('human', 16),
 ('post', 15),
 ('creepy', 14),
 ('bone', 14),
 ('one', 14),
 ('picture', 14),
 ('look', 14),
 ('made', 14),
 ('mask', 14),
 ('skeleton', 13),
 ('friend', 12),
 ('abandoned', 12),
 ('monster', 12),
 ('statue', 12),
 ('x', 12),
 ('wood', 11),
 ('water', 11),
 ('people', 11),
 ('face', 11),
 ('last', 11),
 ('thing', 11),
 ('inside', 10),
 ('back', 10),
 ('house', 10),
 ('dead', 10),
 ('church', 10),
 ('forest', 10),
 ('got', 10),
 ('head', 10),
 ('dark', 10),
 ('museum', 10),
 ('nightmare', 10),
 ('baby', 9),
 ('remains', 9),
 ('teeth', 9),
 ('catacomb', 9),
 ('tree', 9),
 ('boris', 9),
 ('groh', 9),
 ('something', 9),
 ('photo', 9)]

In [36]:
len(subr_text_stopped_freqdist)


2164

In [32]:
total_word_count = sum(subr_text_stopped_freqdist.values())
subr_text_top_50 = subr_text_stopped_freqdist.most_common(50)
print("Word\t\t\tNormalized Frequency")
for word in subr_text_top_50:
    normalized_frequency = word[1] / total_word_count
    print("{} \t\t\t {:.4}".format(word[0], normalized_frequency))

Word			Normalized Frequency
guy 			 0.005921
man 			 0.005702
car 			 0.005482
found 			 0.005044
like 			 0.005044
one 			 0.004386
house 			 0.004167
new 			 0.003947
wtf 			 0.003947
time 			 0.003947
back 			 0.003947
day 			 0.003728
year 			 0.003509
look 			 0.003509
ha 			 0.003289
work 			 0.003289
fire 			 0.003289
fish 			 0.003289
got 			 0.003289
today 			 0.00307
way 			 0.002851
see 			 0.002851
well 			 0.002851
snake 			 0.002851
night 			 0.002632
road 			 0.002632
inside 			 0.002412
think 			 0.002412
going 			 0.002412
baby 			 0.002412
last 			 0.002412
get 			 0.002412
made 			 0.002193
eye 			 0.002193
someone 			 0.002193
head 			 0.002193
driver 			 0.002193
thing 			 0.001974
even 			 0.001974
happen 			 0.001974
fuck 			 0.001974
i'm 			 0.001974
wait 			 0.001974
building 			 0.001974
right 			 0.001974
little 			 0.001974
water 			 0.001974
driving 			 0.001974
go 			 0.001974
live 			 0.001974


In [41]:
bigram_measures = nltk.collocations.BigramAssocMeasures()

In [42]:
subr_text_finder = BigramCollocationFinder.from_words(subr_text_stopped)
subr_text_finder.apply_freq_filter(3)

In [45]:
subr_text_scored = subr_text_finder.score_ngrams(bigram_measures.pmi)
subr_text_scored[:20]

[(('baba', 'yaga'), 10.3409627642517),
 (('concentration', 'camp'), 10.3409627642517),
 (('junji', 'ito'), 10.3409627642517),
 (('stefan', 'koidl'), 10.3409627642517),
 (('p', 'lovecraft'), 9.925925264972856),
 (('czech', 'republic'), 9.3409627642517),
 (('h', 'p'), 8.925925264972854),
 (('boris', 'groh'), 8.756000263530542),
 (('self', 'portrait'), 8.703532843636408),
 (('catacomb', 'paris'), 7.9486453414729406),
 (('x', 'post'), 7.893503787280479),
 (('gas', 'mask'), 7.79664224802789),
 (('real', 'life'), 7.703532843636407),
 (('r', 'pic'), 7.051456147056713),
 (('post', 'r'), 7.019034669364338),
 (('last', 'night'), 6.9519204735058),
 (('look', 'like'), 6.796642248027892),
 (('human', 'remains'), 6.756000263530544),
 (('bone', 'church'), 6.381604748749045),
 (('year', 'old'), 6.3409627642516995)]

# Separating subreddits into 14 classes

In [211]:
import pandas as pd

In [376]:
label_text_df = pd.DataFrame()

In [377]:
label_text_df['target'] = ['Discussion and Stories','Emotional Reaction Fuel','Entertainment - Gaming', 
                         'Entertainment - Television', 'Entertainment - Other (Movies/Music/Franchies/Misc)',
                         'Humor','Images, Gifs, and Videos','Learning and Thinking','Lifestyle and Help',
                         'News and Issues', 'Places', 'Race, Gender, and Identity', 'Sports','Technology']

In [378]:
label_text_df.target

0                                Discussion and Stories
1                               Emotional Reaction Fuel
2                                Entertainment - Gaming
3                            Entertainment - Television
4     Entertainment - Other (Movies/Music/Franchies/...
5                                                 Humor
6                              Images, Gifs, and Videos
7                                 Learning and Thinking
8                                    Lifestyle and Help
9                                       News and Issues
10                                               Places
11                           Race, Gender, and Identity
12                                               Sports
13                                           Technology
Name: target, dtype: object

In [379]:
categorized_posts = dict((i,"")for i in label_text_df.target)
for i in clean_subs:
    if i =="AskReddit" or i=='IAmA' or i =="bestof" or i=='fatpeoplestories' or i =='pettyrevenge' or i =="TalesFromRetail" or i=='DoesAnybodyElse' or i =='CrazyIdeas':
        categorized_posts['Discussion and Stories'] = [categorized_posts['Discussion and Stories'],all_submissions[i]]
    elif i =="WTF" or i=='aww' or i =="cringepics" or i=='JusticePorn' or i =='MorbidReality' or i =="rage" or i=='mildlyinfuriating' or i =='creepy' or i=='creepyPMs' or i =="nosleep" or i=='nostalgia':
        categorized_posts['Emotional Reaction Fuel'] = [categorized_posts['Emotional Reaction Fuel'],all_submissions[i]]
    elif i =="gaming" or i=='leagueoflegends' or i =="pokemon" or i=='Minecraft' or i =='starcraft' or i =="Games" or i=='DotA2' or i =='skyrim' or i=='tf2' or i =="magicTCG" or i=='wow' or i =="KerbalSpaceProgram" or i=='mindcrack' or i =='Fallout' or i=='roosterteeth' or i =="Planetside" or i=='gamegrumps' or i =="battlefield3" or i=='zelda' or i =='darksouls' or i=='masseffect':
        categorized_posts['Entertainment - Gaming'] = [categorized_posts['Entertainment - Gaming'],all_submissions[i]]
    elif i =="arresteddevelopment" or i=='gameofthrones' or i =="doctorwho" or i=='mylittlepony' or i =='community' or i =="breakingbad" or i=='adventuretime' or i =='startrek' or i=='TheSimpsons' or i =="futurama" or i=='HIMYM' or i=='DunderMifflin' or i =="thewalkingdead":
        categorized_posts['Entertainment - Television'] = [categorized_posts['Entertainment - Television'],all_submissions[i]]
    elif i =="Music" or i=='movies' or i =="harrypotter" or i=='StarWars' or i =='DaftPunk' or i =="hiphopheads" or i=='anime' or i =='comicbooks' or i=='geek' or i =="batman" or i=='TheLastAirbender' or i=='Naruto' or i =="FanTheories":
        categorized_posts['Entertainment - Other (Movies/Music/Franchies/Misc)'] = [categorized_posts['Entertainment - Other (Movies/Music/Franchies/Misc)'],all_submissions[i]]
    elif i =="funny" or i=='AdviceAnimals' or i =="fffffffuuuuuuuuuuuu" or i=='4chan' or i =='ImGoingToHellForThis' or i =="firstworldanarchists" or i=='circlejerk' or i =='MURICA' or i=='facepalm' or i =="Jokes" or i=='wheredidthesodago' or i=='polandball' or i =="TrollXChromosomes" or i =="comics" or i=='nottheonion' or i=='britishproblems' or i =="TumblrInAction" or i =="onetruegod":
        categorized_posts['Humor'] = [categorized_posts['Humor'],all_submissions[i]]
    elif i =="pics" or i=='videos' or i =="gifs" or i=='reactiongifs' or i =='mildlyinteresting' or i =="woahdude" or i=='FiftyFifty' or i =='FoodPorn' or i=='HistoryPorn' or i =="wallpapers" or i=='youtubehaiku' or i=='Unexpected' or i =="photoshopbattles" or i =="AnimalsBeingJerks" or i=='cosplay' or i=='EarthPorn' or i =="QuotesPorn" or i =="awwnime" or i =="AbandonedPorn" or i=='carporn' or i =="PerfectTiming" or i=='OldSchoolCool' or i =='RoomPorn' or i =="woahdude" or i=='Pareidolia' or i =='MapPorn' or i=='tumblr' or i =="techsupportgore" or i=='PrettyGirls' or i=='itookapicture':
        categorized_posts['Images, Gifs, and Videos'] = [categorized_posts['Images, Gifs, and Videos'],all_submissions[i]]
    elif i =="todayilearned" or i=='science' or i =="askscience" or i=='space' or i =='AskHistorians' or i =="YouShouldKnow" or i=='explainlikeimfive':
        categorized_posts['Learning and Thinking'] = [categorized_posts['Learning and Thinking'],all_submissions[i]]
    elif i =="trees" or i=='MakeupAddiction' or i =="cats" or i=='LifeProTips' or i =='RedditLaqueristas' or i =="Random_Acts_Of_Amazon" or i=='food' or i =='guns' or i=='tattoos' or i =="corgi" or i=='teenagers' or i =='GetMotivated' or i=='motorcycles' or i =="sex" or i=='progresspics' or i =='DIY' or i=='bicycling' or i =="Fitness" or i=='lifehacks' or i =='longboarding' or i=='Frugal' or i =="drunk" or i=='Art' or i=='loseit' or i =='Military':
        categorized_posts['Lifestyle and Help'] = [categorized_posts['Lifestyle and Help'],all_submissions[i]]
    elif i =="politics" or i=='worldnews' or i =="news" or i=='conspiracy' or i =='Libertarian' or i =="TrueReddit" or i=='Conservative' or i=='offbeat':
        categorized_posts['News and Issues'] = [categorized_posts['News and Issues'],all_submissions[i]]
    elif i =="canada" or i=='toronto' or i =="australia" or i=='unitedkingdom':
        categorized_posts['Places'] = [categorized_posts['Places'],all_submissions[i]]
    elif i =="atheism" or i=='TwoXChromosomes' or i =="MensRights" or i=='gaybros' or i =='lgbt':
        categorized_posts['Race, Gender, and Identity'] = [categorized_posts['Race, Gender, and Identity'],all_submissions[i]]
    elif i =="nba" or i=='soccer' or i =="hockey" or i=='nfl' or i =='formula1' or i =="baseball" or i=='MMA' or i=='SquaredCircle':
        categorized_posts['Sports'] = [categorized_posts['Sports'],all_submissions[i]]
    elif i =="technology" or i=='Android' or i =="Bitcoin" or i=='programming' or i =='apple':
        categorized_posts['Technology'] = [categorized_posts['Technology'],all_submissions[i]]

In [380]:
def flatten(container):
    for i in container:
        if isinstance(i, (list,tuple)):
            for j in flatten(i):
                yield j
        else:
            yield i

In [381]:
posts_by_category = dict((i,"")for i in label_text_df.target)
for i in label_text_df.target:
    posts_by_category[i] = list(flatten(categorized_posts[i]))[1:]

In [382]:
posts_by_category['Discussion and Stories']

["People who haven't pooped in 2019 yet, why are you still holding on to last years shit?",
 'Stan Lee has passed away at 95 years old',
 'How would you feel about a feature where if someone upvotes a crosspost, the original post is upvoted automatically?',
 'Reddit, how would you feel about a law that bans radio stations from playing commercials with honking/beeping/siren noises in them?',
 'Without saying what the category is, what are your top five?',
 'Anthony Bourdain once said "There\'s a guy in my head, and all he wants to do is lay in bed all day long, smoke pot, and watch old movies and cartoons. My life is a series of strategems, to avoid, and outwit that guy". Who is "that guy" for you, and what do you do to avoid him?',
 'With all of the negative headlines dominating the news these days, it can be difficult to spot signs of progress. What makes you optimistic about the future?',
 '[Serious] Americans, would you be in support of putting a law in place that government officia

In [383]:
master_list = []
for i in tqdm(label_text_df.target):
    for post in posts_by_category[i]:
        post_list = []
        post_list.append(i)
        post_list.append(post)
        master_list.append(post_list)
        
#     posts_by_category[i] = " ".join(posts_by_category[i])

In [385]:
master_list[0]

['Discussion and Stories',
 "People who haven't pooped in 2019 yet, why are you still holding on to last years shit?"]

In [392]:
reddit_df = pd.DataFrame(master_list)

In [393]:
reddit_df.to_csv('reddit_df')

In [400]:
reddit_df.head(2)

,0,1
0,Discussion and Stories,"People who haven't pooped in 2019 yet, why are..."
1,Discussion and Stories,Stan Lee has passed away at 95 years old


### Using GloVes multi-dimensional word vectorizer to build a model

In [402]:
target = reddit_df[0]
data = reddit_df[1].map(word_tokenize).values

In [404]:
total_vocabulary = set(word for headline in data for word in headline)
print("There are {} unique tokens in the dataset.".format(len(total_vocabulary)))

There are 113827 unique tokens in the dataset.


In [407]:
glove = {}
with open('glove.6B.100d.txt', 'rb') as f:
    for line in f:
        parts = line.split()
        word = parts[0].decode('utf-8')
        if word in total_vocabulary:
            vector = np.array(parts[1:], dtype=np.float32)
            glove[word] = vector

In [408]:
class W2vVectorizer(object):
    
    def __init__(self, w2v):
        # takes in a dictionary of words and vectors as input
        self.w2v = w2v
        if len(w2v) == 0:
            self.dimensions = 0
        else:
            self.dimensions = len(w2v[next(iter(glove))])
    
    # Note from Mike: Even though it doesn't do anything, it's required that this object implement a fit method or else
    # It can't be used in a sklearn Pipeline. 
    def fit(self, X, y):
        return self
            
    def transform(self, X):
        return np.array([
            np.mean([self.w2v[w] for w in words if w in self.w2v]
                   or [np.zeros(self.dimensions)], axis=0) for words in X])

In [410]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

rf =  Pipeline([("Word2Vec Vectorizer", W2vVectorizer(glove)),
              ("Random Forest", RandomForestClassifier(n_estimators=100, verbose=True))])
svc = Pipeline([("Word2Vec Vectorizer", W2vVectorizer(glove)),
                ('Support Vector Machine', SVC())])
lr = Pipeline([("Word2Vec Vectorizer", W2vVectorizer(glove)),
              ('Logistic Regression', LogisticRegression())])
models = [('Random Forest', rf),
          ("Support Vector Machine", svc),
          ("Logistic Regression", lr)]

In [412]:
scores = [(name, cross_val_score(model, data, target, cv=2).mean()) for name, model, in tqdm(models)]

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  1.7min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    3.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  1.7min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    3.2s finished
/Users/reuben/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/reuben/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: Fut

In [413]:
scores

[('Random Forest', 0.2268008048957078),
 ('Support Vector Machine', 0.24213128541066958),
 ('Logistic Regression', 0.2312619396923426)]

#### ^^^ Initial results seem low, but, with 14 categories, random guessing would achieve an accuracy of 7.1%. Let's try a Deep Neural Network with and Embedding Layer

In [414]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Dense, LSTM, Embedding
from keras.layers import Dropout, Activation, Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.preprocessing import text, sequence

Using TensorFlow backend.


In [415]:
y = pd.get_dummies(target).values

In [419]:
tokenizer = text.Tokenizer(num_words=20000)
tokenizer.fit_on_texts(list(reddit_df[1]))
list_tokenized_headlines = tokenizer.texts_to_sequences(reddit_df[1])
X_t = sequence.pad_sequences(list_tokenized_headlines, maxlen=100)

In [451]:
embedding_size = 128
input_ = Input(shape=(100,))
x = Embedding(20000, embedding_size)(input_)
x = LSTM(25, return_sequences=True)(x)
x = GlobalMaxPool1D()(x)
x = Dropout(0.5)(x)
x = Dense(50, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(50, activation='relu')(x)
x = Dense(50, activation='relu')(x)
x = Dense(50, activation='relu')(x)
# There are 14 different possible classes, so we use 14 neurons in our output layer
x = Dense(14, activation='softmax')(x)

model = Model(inputs=input_, outputs=x)

In [452]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [453]:
model.summary()

Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding_8 (Embedding)      (None, 100, 128)          2560000   
_________________________________________________________________
lstm_8 (LSTM)                (None, 100, 25)           15400     
_________________________________________________________________
global_max_pooling1d_8 (Glob (None, 25)                0         
_________________________________________________________________
dropout_14 (Dropout)         (None, 25)                0         
_________________________________________________________________
dense_14 (Dense)             (None, 50)                1300      
_________________________________________________________________
dropout_15 (Dropout)         (None, 50)                0   

In [455]:
model.fit(X_t, y, epochs=10, batch_size=128, validation_split=0.2)

Train on 138444 samples, validate on 34611 samples
Epoch 1/10
138444/138444 [==============================] - 159s 1ms/step - loss: 1.8946 - accuracy: 0.2795 - val_loss: 8.2535 - val_accuracy: 0.0353
Epoch 2/10
138444/138444 [==============================] - 159s 1ms/step - loss: 1.5115 - accuracy: 0.4599 - val_loss: 10.4834 - val_accuracy: 0.0379
Epoch 3/10
  1920/138444 [..............................] - ETA: 2:30 - loss: 1.3633 - accuracy: 0.5333

KeyboardInterrupt: 

In [41]:
def tokenize(text):
    combined_titles = ""
    for title in text:
        combined_titles += title + " "
    return word_tokenize(combined_titles)


In [42]:
def count_vectorize(text, vocab=None):
    if vocab:
        unique_words = vocab
    else:
        unique_words = list(set(text))
    
    word_dict = {i:0 for i in unique_words}
    
    for word in text:
        word_dict[word] += 1
    
    return word_dict


In [44]:
for sub in clean_subs:
    tokenized_words = tokenize(all_submissions[sub])
    counted_words = count_vectorize(tokenized_words)
    print(counted_words)
    

{'all': 3, 'law': 3, 'top': 1, 'for': 1, 'makes': 1, 'them': 1, 'about': 5, 'last': 1, 'such': 1, 'upvoted': 1, 'shutdowns': 1, 'people': 1, 'signs': 1, 'the': 8, 'future': 1, 'specialized': 1, '[': 1, 'your': 1, 'days': 1, 'go': 1, 'employees': 1, 'become': 1, 'What': 1, 'stations': 1, 'do': 5, 'wants': 1, 'original': 1, 'other': 1, 'putting': 1, 'Americans': 1, 'series': 1, 'guy': 3, 'be': 2, 'still': 1, 'my': 1, 'pass': 1, 'what': 3, 'a': 8, 'cartoons': 1, 'strategems': 1, 'with': 2, 'not': 1, 'while': 1, 'People': 1, 'radio': 1, 'Iceland': 1, 'over': 2, 'Anthony': 1, 'every': 1, 'where': 1, 'someone': 1, 'noises': 1, 'life': 1, 'donors': 1, '18': 1, 'test': 1, 'of': 6, 'news': 1, "'s": 1, 'lay': 1, '95': 1, 'difficult': 1, 'pot': 1, 'as': 1, 'federal': 1, 'like': 1, 'automatically': 2, 'five': 1, 'Bourdain': 1, 'smoke': 1, 'category': 1, 'commercials': 1, 'crosspost': 1, 'in': 7, 'these': 1, 'requires': 1, 'why': 2, 'pooped': 1, 'years': 2, 'Lee': 1, 'How': 3, 'government': 1, '201

{'against': 1, 'law': 1, 'for': 1, 'clients': 1, 'DOM': 1, 'hard': 1, 'implemented': 1, 'alternative': 1, 'admit': 1, 'tens': 1, 'called': 1, 'PR': 1, 'free': 1, 'people': 1, 'Polymer': 1, 'the': 5, 'wrong': 1, 'They': 1, 'spent': 1, 'v0': 1, '(': 2, 'did': 1, 'JavaScript': 1, 'start': 1, 'algorithms': 1, 'Bounty': 1, 'The': 1, 'Paid': 1, 'WPF': 1, 'somebody': 1, 'really': 1, 'character': 1, 'programming': 1, 'my': 2, 'a': 7, 'Edge': 1, 'always': 1, 'work': 1, 'programmers': 1, 'read': 1, '$': 1, 'Bug': 1, 'open': 1, 's': 2, 'x-post': 1, 'OptiKey': 1, 'IKEA': 1, 'over': 1, 'probably': 1, 'load': 1, 'slower': 1, 'where': 1, 'tracking': 1, 'reason': 2, 'redesign': 1, 'of': 8, 'explains': 1, 'Google': 2, "'s": 3, 'CPU': 1, 'their': 1, 'as': 3, 'interesting': 1, 'Intel': 1, '“': 1, 'Got': 1, 'TIL': 1, 'dwitter': 1, 'instruction': 1, 'spare': 1, '140': 1, 'YouTube': 2, 'inside': 1, 'mess': 2, 'in': 4, 'compose': 1, 'patenting': 1, ':': 3, 'harder': 1, '5x': 1, 'needs': 1, 'software': 1, 'ye

In [80]:
def term_frequency(BoW_dict):
    total_word_count = sum(BoW_dict.values())
    for ind, val in BoW_dict.items():
        BoW_dict[ind] = val/ total_word_count
    
    return BoW_dict   

In [126]:
def inverse_document_frequency(list_of_dicts):
    vocab_set = set()
    # Iterate through list of dfs and add index to vocab_set
    for d in list_of_dicts:
        for word in d.keys():
            vocab_set.add(word)
    
    # Once vocab set is complete, create an empty dictionary with a key for each word and value of 0.
    full_vocab_dict = {i:0 for i in vocab_set}
    
    # Loop through each word in full_vocab_dict
    for word, val in full_vocab_dict.items():
        docs = 0
        
        # Loop through list of dicts.  Each time a dictionary contains the word, increment docs by 1
        for d in list_of_dicts:
            if word in d:
                docs += 1
        
        # Now that we know denominator for equation, compute and set IDF value for word
        
        full_vocab_dict[word] = np.log((len(list_of_dicts)/ float(docs)))
    
    return full_vocab_dict

In [127]:
def tf_idf(list_of_dicts):
    # Create empty dictionary containing full vocabulary of entire corpus
    doc_tf_idf = {}
    idf = inverse_document_frequency(list_of_dicts)
    full_vocab_list = {i:0 for i in list(idf.keys())}
    
    # Create tf-idf list of dictionaries, containing a dictionary that will be updated for each document
    tf_idf_list_of_dicts = []
    
    # Now, compute tf and then use this to compute and set tf-idf values for each document
    for doc in list_of_dicts:
        doc_tf = term_frequency(doc)
        for word in doc_tf:
            doc_tf_idf[word] = doc_tf[word] * idf[word]
        tf_idf_list_of_dicts.append(doc_tf_idf)
    
    return tf_idf_list_of_dicts

In [140]:
def main(subreddits):
    vectorized_dicts = []
    for sub in subreddits:
        # Clean and tokenize raw text
        titles = all_submissions[sub]
        tokenized = tokenize(titles)
        
        # Get count vectorized representation and store in vectorized_dicts  
        count_vectorized_document = count_vectorize(tokenized)
        vectorized_dicts.append(count_vectorized_document)
    tf_idf_all_docs = tf_idf(vectorized_dicts)
    return tf_idf_all_docs

tf_idf_all_subs = main(clean_subs)
print(list(tf_idf_all_subs[0])[:10])

['all', 'law', 'top', 'for', 'makes', 'them', 'about', 'last', 'such', 'upvoted']


In [141]:
num_dims = len(tf_idf_all_subs[0])
print("Number of Dimensions: {}".format(num_dims))

Number of Dimensions: 7246


In [143]:
tf_idf_vals_list = []

for i in tf_idf_all_subs:
    tf_idf_vals_list.append(list(i.values()))
    
tf_idf_vals_list[0][:10]

[0.0063899645979554566,
 0.013238089526409112,
 0.022549467454425442,
 0.00810829994634717,
 0.021935876503284615,
 0.00855968495688575,
 0.013680404231525679,
 0.02219134409677715,
 0.010794647047316603,
 0.04568711035181843]

In [144]:
from sklearn.manifold import TSNE

t_sne_object_3d = TSNE(n_components=3)
transformed_data_3d = t_sne_object_3d.fit_transform(tf_idf_vals_list)
transformed_data_3d

array([[-1.48992020e+02,  2.71296940e+01, -1.44119308e+02],
       [-1.01203056e+02,  1.37120255e+02, -1.11903122e+02],
       [-1.60467300e+02, -7.76967163e+01, -1.07780054e-01],
       [-1.60467300e+02, -7.76967163e+01, -1.07780054e-01],
       [-7.68352509e+01,  3.73754921e+01,  1.14591057e+02],
       [-4.48668060e+01,  1.73314270e+02,  8.56712036e+01],
       [ 1.53636887e+02, -6.89396667e+01, -1.05651833e+02],
       [ 1.15672256e+02, -6.78719406e+01,  1.25600100e+01],
       [-2.09356827e+02, -2.08444357e+00,  3.38676987e+01],
       [-7.44169083e+01, -6.30411720e+01,  1.39170792e+02],
       [ 7.66209106e+01, -1.18204165e+01,  4.63080864e+01],
       [-1.05603928e+02,  1.47474197e+02,  2.10921745e+01],
       [-7.44169083e+01, -6.30411720e+01,  1.39170792e+02],
       [-9.26156311e+01, -7.91694489e+01, -5.32216988e+01],
       [-6.60036240e+01, -1.25242867e+02,  5.49799652e+01],
       [ 8.74106674e+01, -1.10726410e+02, -6.04048004e+01],
       [-5.01531982e+01,  4.07699299e+00